In [10]:
# three phases:
# 4 - slow depolarization
# 0 - rapid depolarization
# 3 - repolarization

# three types of variables:

# t0 - duration of phase 0
# t3 - duration of phase 3
# t4 - duration of phase 4

# t40 - time of shift from phase 4 to phase 0
# t03 - time of shift from phase 3 to phase 0
# t34 - time of shift from phase 3 to phase 4

# V40 - potential at t40 (threshold potential that triggers the rapid depolarization phase)
# V03 - potential at t03 (peak potential)
# V34 - potential at t34 ("resting" membrane potential - the lowest potential)

# translation from variables to sliders:

# V40 - Threshold Potential
# V03 - Peak Membrane Potential
# V34 - Maximum Diastolic Potential

# t0 - Rapid Depolarization Duration
# t3 - Repolarizarion Duration
# t4 - Slow Depolarization Duration

In [11]:
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interact, interactive, FloatSlider, IntSlider, Dropdown

In [12]:
def smooth_transition(x, x0, eps=0.005):
    """
    Sigmoid-based transition function.
    Returns values smoothly varying from 0 to 1 around x0.
    
    eps controls the "width" of the transition region.
    Smaller eps = sharper transition.
    """
    return 1.0 / (1.0 + np.exp(-(x - x0)/eps))

def sigmoid(t, V_lo, V_hi, t0, k):
    """
    Sigmoid-based time-voltage function for potential curve fitting.
    Used for phase 0 estimation (rapid depolarization).
    """
    return V_lo + (V_hi - V_lo) / (1 + np.exp(-(t - t0)/k))

In [13]:
def pacemaker_AP(t,
                 V34, V40, V03, 
                 t03, t34, t40,
                 eps=0.01):
    """
    Phase 0: sigmoid
    Phase 3: cubic (zero slope at peak and end)
    Phase 4: linear
    Smoothly blended at t_thresh and t_peak.
    """

    # Phase 0 - rapid depolarization
    t0_phase0 = 0.5*(t40 + t03)
    k_phase0 = (t03 - t40)/6.0
    P0 = sigmoid(t, V40, V03, t0_phase0, k_phase0)

    # Phase 3 - repolarization
    t3 = t - t03
    T3 = t34 - t03
    # Hermite-like cubic: V(0)=V03, V(T3)=V34, V'(0)=0, V'(T3)=0
    # Standard solution:
    # V3(t3) = V_peak*(2*(t3/T3)^3 - 3*(t3/T3)^2 + 1) + V_rest*(-2*(t3/T3)^3 + 3*(t3/T3)^2)
    u = np.clip(t3 / max(T3, 1e-12), 0.0, 1.0)
    P3 = V03*(2*u**3 - 3*u**2 + 1) + V34*(-2*u**3 + 3*u**2)

    # Phase 4 - slow depolarization
    slope4 = (V40 - V34) / t40
    P4 = V34 + slope4 * t

    # Smooth blending
    w40 = smooth_transition(t, t40, eps)
    w03 = smooth_transition(t, t03, eps)
    V = (1 - w40)*P4 + w40*((1 - w03)*P0 + w03*P3)
    return V

In [ ]:
def interactive_pacemaker(V34, V40, V03,
                          t4, t0, t3,
                          n_cycles, 
                          eps=0.005):
    
    t4, t0, t3 = (x / 1000 for x in (t4, t0, t3))

    t40 = t4
    t03 = t4 + t0
    t34 = t4 + t0 + t3

    # Time axis
    t_cycle = np.linspace(0, t34, 600)
    V_cycle = pacemaker_AP(t_cycle,
                                V34, V40, V03,
                                t03, t34, t40, 
                                eps=eps)
    # Repeat cycles
    t_total = np.linspace(0, n_cycles*t34, n_cycles*len(t_cycle))
    V_total = np.tile(V_cycle, n_cycles)

    # Plot
    plt.figure(figsize=(9,4))
    plt.plot(t_total, V_total, lw=2)
    plt.xlabel("Time (s)")
    plt.ylabel("Voltage (mV)")
    plt.title(f"Simulated Pacemaker Action Potential ({n_cycles} cycles)")
    plt.grid(True)
    plt.ylim(V34-10, V03+10)
    plt.show()


In [20]:
slider_layout = {'width': '600px'} 
slider_style = {'description_width': '220px'}

In [21]:
presets = {
    "SA node": dict(
        V34=-60,
        V40=-40,
        V03=10,
        t4=120,
        t0=8,
        t3=150
    ),
    "AV node": dict(
        V34=-65,
        V40=-45,
        V03=5,
        t4=220,
        t0=10,
        t3=180
    )
}

In [22]:
preset_dropdown = Dropdown(
    options=["SA node", "AV node"],
    value="SA node",
    description="Preset:"
)

In [ ]:
def update_sliders(preset):
    p = presets[preset]
    interact(
        interactive_pacemaker,
        V34=FloatSlider(value=p["V34"], min=-80, max=-40, step=2,
                        description="Maximum Diastolic Potential (mV)", 
                        style=slider_style, layout=slider_layout),
        V40=FloatSlider(value=p["V40"], min=-60, max=-20, step=2,
                        description="Threshold Potential (mV)", 
                        style=slider_style, layout=slider_layout),
        V03=FloatSlider(value=p["V03"], min=-10, max=40, step=2,
                        description="Peak Membrane Potential (mV)", 
                        style=slider_style, layout=slider_layout),
        t4=FloatSlider(value=p["t4"], min=50, max=400, step=10,
                       description="Slow Depolarization Duration (ms)", 
                       style=slider_style, layout=slider_layout),
        t0=FloatSlider(value=p["t0"], min=2, max=20, step=1,
                       description="Rapid Depolarization Duration (ms)", 
                       style=slider_style, layout=slider_layout),
        t3=FloatSlider(value=p["t3"], min=50, max=300, step=10,
                       description="Repolarization Duration (ms)", 
                       style=slider_style, layout=slider_layout),
        n_cycles=IntSlider(value=2, min=1, max=5, step=1,
                           description="Cycles", 
                           style=slider_style, layout=slider_layout),
        eps=FloatSlider(value=0.01, min=0.005, max=0.05, step=0.005,
                        description="Blend width",
                        style=slider_style, layout=slider_layout)
    )

interactive(update_sliders, preset=preset_dropdown)

interactive(children=(Dropdown(description='Preset:', options=('SA node', 'AV node'), value='SA node'), Output…